In [3]:
import datetime
import pandas as pd
import numpy as np
import argparse
import sys

sys.path.append('../..')
from utils.func import norm_time

In [5]:
parser = argparse.ArgumentParser()
parser.add_argument('--train_date', type=int, default=2014)
parser.add_argument('--test_date', type=int, default=2015)
parser.add_argument('--code', type=list, default=['7203'])
# parser.add_argument('--code', type=list, default=['7203', '9984'])
# parser.add_argument('--code', type=list, default=['8301', '7203', '9501', '6758', '9984',
#                                                  '8306', '8411', '6501', '6752', '6502'])

args = parser.parse_args(args=[])

code_list = [str(x) for x in args.code]

for i, date in enumerate(range(2011, 2018)):
    tmp = pd.read_csv('../../data/news/' + str(date) + '.csv', encoding='cp932')
    tmp = tmp[tmp['Company_IDs(TSE)'].isin(code_list)]
    # tmp = tmp[tmp['Company_Relevance'] == str(100)]
    tmp = tmp[['Time_Stamp_Original(JST)', 
                    'Company_Code(TSE)', 
                    'Headline', 
                    'News_Source',
                    'Company_Relevance', 
                    'Keyword_Article']]

    # 欠損除去
    tmp = tmp[~tmp["Keyword_Article"].isnull()]

    # タグ除去
    tmp = tmp[(tmp['News_Source'] == '日経') | 
                 (tmp['News_Source'] == 'ＮＱＮ') |
                 (tmp['News_Source'] == 'ＱＵＩＣＫ') | 
                 (tmp['News_Source'] == 'Ｒ＆Ｉ')]

    tmp['code'] = tmp['Company_Code(TSE)'].astype(int)
    tmp['date'] = pd.to_datetime(tmp["Time_Stamp_Original(JST)"]).map(norm_time)
    tmp = tmp.set_index(['date', 'code'], drop=True)
    tmp = tmp.drop(['Time_Stamp_Original(JST)', 'Company_Code(TSE)'], axis=1)

    if i == 0:
        df1 = tmp.copy()
    else:
        df1 = pd.concat([df1, tmp])

In [7]:
# 株価を取り出す
for i, code in enumerate(code_list):
    tmp = pd.read_csv('../../data/stock_price/' + str(code) + '.csv', index_col=0)
    tmp['code'] = int(code)
    if i == 0:
        df2 = tmp
    else:
        df2 = pd.concat([df2, tmp])

df2['date'] = pd.to_datetime(df2['date'])
df2 = df2.set_index(['date', 'code'], drop=True)
print(df2.shape)

(2175, 1)


In [8]:
# 時系列をくっつける
df3 = pd.concat([df1, df2], axis=1, join_axes=[df1.index], levels=[0, 1])
df3 = df3.sort_values(by=['code', 'date'])
df3['Keyword_Article'] = \
df3.groupby(level=[0,1]).apply(lambda x: ':<pad>:'.join(list(x['Keyword_Article'])))

df3 = df3.dropna()
df3 = df3[~df3.duplicated(subset=['Keyword_Article'])]
df3['price'] = \
     df3['adj_close'].groupby(level=['code']).pct_change(1).shift(-1)*100
df3 = df3.dropna()

/home/ts-zemi/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.
  


In [9]:
df3

,,Headline,News_Source,Company_Relevance,Keyword_Article,adj_close,price
date,code,,,,,,
2011-01-05,7203,<日経>◇次世代車の研究開発 名大に国内最大拠点,日経,38,安全:環境:負荷:開発:目指す:開所式:研究拠点:効率:簡素化:次世代:電気自動車:電気:幅...,3295.0,2.579666
2011-01-06,7203,<日経>◇日本経済新聞５日夕刊早版,日経,10,加速:環境:増:後退:貢献:債券:投資:温暖化対策:ワクチン:マグロ:初セリ:円相場:反発:...,3380.0,2.218935
2011-01-07,7203,<NQN>◇トヨタ社長「今年は後半に晴れ間」 為替は１ドル＝90円を期待,ＮＱＮ,100,豊田:見通し:販売:エコカー補助金:安定的:伸び:株価:為替:水準:日経平均株価:最低:ライ...,3455.0,0.000000
2011-01-11,7203,<日経>◇ＧＭ、ビュイック初の「高級小型車」を発表,日経,36,開幕:北米:ブランド:小型車:新型車:高級車:高い:高級ブランド:<pad>:開幕:位置付け...,3455.0,1.302460
2011-01-12,7203,<日経>◇トヨタ、米ミシガン州に安全研究センター新設,日経,100,先進:安全:子供:高齢者:事故:向上:目指す:米国:大規模:リコール:回収:問題:開催:豊田...,3500.0,1.000000
...,...,...,...,...,...,...,...
2017-12-21,7203,<日経>◇トヨタ、タイでＥＶシェアサービス開始 19年末の商用化目指す,日経,100,バンコク:タイ:電気自動車:ＥＶ:サービス:首都:導入:大学:運用:範囲:拡大:目指す:バー...,7288.0,-0.082327
2017-12-22,7203,<日経>◇トヨタ、17年の世界販売、過去最高の1035万台,日経,100,生産:販売実績:全体:台数:過去最高:新型車:販売:好調:措置:延長:受け:中国:小型車:伸...,7282.0,-0.398242
2017-12-26,7203,<日経>◇道産子ＥＶ「ネイクル」、２人乗りを開発へ,日経,32,中小:北海道:本部:コンソーシアム:寒冷地:電気自動車:ＥＶ:開発:バイク:ウレタン:外装:...,7253.0,0.289535


In [10]:

# # CSVファイルに保存する
# df4 = pd.concat([df3[['Keyword_Article', 'price']].rename(
#                                    columns={'Keyword_Article': 'state', 'price': 'reward'}),
#                              df3[['Keyword_Article']].shift(-1).rename(
#                                    columns={'Keyword_Article': 'next_state'})], axis=1).dropna()
# df4 = df4[['state', 'next_state', 'reward']]

# date_year = df4.index.map(lambda x: x[0].year)

# df4[date_year <= args.train_date].to_csv(
#     './data/news/text_train.tsv',
#     header=None,
#     index=None,
#     sep='\t')

# df4[(args.train_date < date_year) & (date_year < args.test_date)].to_csv(
#     './data/news/text_val.tsv',
#     header=None,
#     index=None,
#     sep='\t')

# df4[(args.test_date <= date_year) & (date_year <= 2017)].to_csv(
#     './data/news/text_test.tsv',
#     header=None,
#     index=None,
#     sep='\t')
